In [1]:
%load_ext rpy2.ipython

C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  "The symbol '%s' is not in this R namespace/package." % name


In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import urllib

In [3]:
%%R
data_folder = './'
setwd(data_folder)

In [4]:
out_msstats = pd.read_csv("../PXD007145-Th.sdrf_openms_design_msstats_in.csv", index_col=None)

import re
def sub_mod(peptide):
    peptide = peptide.replace(".", "")
    peptide = re.sub("\(.*?\)", "", peptide)
    return peptide
    
out_msstats["sequence"] = out_msstats.apply(lambda x: sub_mod(x["PeptideSequence"]), axis=1)

print(len(set(out_msstats["sequence"])))
print(len(set(out_msstats["ProteinName"])))


out_msstats = pd.read_csv("../PXD007145-Th.sdrf_openms_design_msstats_in.csv", index_col=None)
out_msstats["sequence"] = out_msstats.apply(lambda x: sub_mod(x["PeptideSequence"]), axis=1)
filtered_df = out_msstats.groupby('sequence').filter(lambda x: len(set(x["ProteinName"])) == 1)
filtered_df = filtered_df.groupby('ProteinName').filter(lambda x: len(set(x["sequence"])) >= 2)
filtered_df = filtered_df[-filtered_df["ProteinName"].str.contains(";")]

print(len(set(filtered_df["sequence"])))
print(len(set(filtered_df["ProteinName"])))

filtered_df.drop("sequence", axis=1, inplace=True)
filtered_df.to_csv("../PXD007145-Th_filter.sdrf_openms_design_msstats_in.csv", index=False)

14247
5219
11785
2829


In [5]:
%%R -w 800 -h 600
library(MSstats)
library(reticulate)

# quantms
raw.om <- read.csv('../PXD007145-Th_filter.sdrf_openms_design_msstats_in.csv')
raw.om <- raw.om[!grepl("DECOY", raw.om$ProteinName),]
fileData <- OpenMStoMSstatsFormat(raw.om)

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = 'equalizeMedians',
                                         summaryMethod = 'TMP',
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------

colnames(ourMatrix) <- ourCondition

DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_quantms_EM_NaN_output_filter.csv")

From cffi callback <function _consolewrite_ex at 0x000001DE4210E828>:
Traceback (most recent call last):
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 132, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 133, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 1: invalid continuation byte
R[write to console]: The following object is masked from 'package:grDevices':

    savePlot




INFO  [2023-01-07 16:13:32] ** Raw data from OpenMS imported successfully.
INFO  [2023-01-07 16:13:32] ** Raw data from OpenMS cleaned successfully.
INFO  [2023-01-07 16:13:32] ** Using annotation extracted from quantification data.
INFO  [2023-01-07 16:13:32] ** Run labels were standardized to remove symbols such as '.' or '%'.
INFO  [2023-01-07 16:13:32] ** The following options are used:
  - Features will be defined by the columns: PeptideSequence, PrecursorCharge, FragmentIon, ProductCharge
  - Shared peptides will be removed.
  - Proteins with single feature will not be removed.
  - Features with less than 3 measurements across runs will be removed.
INFO  [2023-01-07 16:13:33] ** Features with all missing measurements across runs are removed.
INFO  [2023-01-07 16:13:33] ** Shared peptides are removed.
INFO  [2023-01-07 16:13:33] ** Multiple measurements in a feature and a run are summarized by summaryforMultipleRows: max
INFO  [2023-01-07 16:13:33] ** Features with one or two meas

## quantms

In [6]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_EM_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=4-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

537
91
1958
154
PPV = 0.855096
NPV = 0.927083


In [10]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_EM_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=10-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

608
373
1676
83
PPV = 0.619776
NPV = 0.952814


In [11]:
%%R -w 800 -h 600
library(MSstats)
library(reticulate)

# quantms
raw.om <- read.csv('../PXD007145-Th_filter.sdrf_openms_design_msstats_in.csv')
raw.om <- raw.om[!grepl("DECOY", raw.om$ProteinName),]
fileData <- OpenMStoMSstatsFormat(raw.om)

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = 'quantile',
                                         summaryMethod = 'TMP',
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------

colnames(ourMatrix) <- ourCondition

DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_quantms_Q_NaN_output_filter.csv")

INFO  [2023-01-07 16:22:32] ** Raw data from OpenMS imported successfully.
INFO  [2023-01-07 16:22:32] ** Raw data from OpenMS cleaned successfully.
INFO  [2023-01-07 16:22:32] ** Using annotation extracted from quantification data.
INFO  [2023-01-07 16:22:32] ** Run labels were standardized to remove symbols such as '.' or '%'.
INFO  [2023-01-07 16:22:32] ** The following options are used:
  - Features will be defined by the columns: PeptideSequence, PrecursorCharge, FragmentIon, ProductCharge
  - Shared peptides will be removed.
  - Proteins with single feature will not be removed.
  - Features with less than 3 measurements across runs will be removed.
INFO  [2023-01-07 16:22:32] ** Features with all missing measurements across runs are removed.
INFO  [2023-01-07 16:22:32] ** Shared peptides are removed.
INFO  [2023-01-07 16:22:33] ** Multiple measurements in a feature and a run are summarized by summaryforMultipleRows: max
INFO  [2023-01-07 16:22:33] ** Features with one or two meas

In [12]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_Q_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=4-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

536
92
1957
155
PPV = 0.853503
NPV = 0.926610


In [15]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_Q_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=10-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

609
378
1671
82
PPV = 0.617021
NPV = 0.953223


In [16]:
%%R -w 800 -h 600
library(MSstats)
library(reticulate)

# quantms
raw.om <- read.csv('../PXD007145-Th_filter.sdrf_openms_design_msstats_in.csv')
raw.om <- raw.om[!grepl("DECOY", raw.om$ProteinName),]
fileData <- OpenMStoMSstatsFormat(raw.om)

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = FALSE,
                                         summaryMethod = 'TMP',
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------

colnames(ourMatrix) <- ourCondition

DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_quantms_NN_NaN_output_filter.csv")

INFO  [2023-01-07 16:33:15] ** Raw data from OpenMS imported successfully.
INFO  [2023-01-07 16:33:15] ** Raw data from OpenMS cleaned successfully.
INFO  [2023-01-07 16:33:15] ** Using annotation extracted from quantification data.
INFO  [2023-01-07 16:33:15] ** Run labels were standardized to remove symbols such as '.' or '%'.
INFO  [2023-01-07 16:33:15] ** The following options are used:
  - Features will be defined by the columns: PeptideSequence, PrecursorCharge, FragmentIon, ProductCharge
  - Shared peptides will be removed.
  - Proteins with single feature will not be removed.
  - Features with less than 3 measurements across runs will be removed.
INFO  [2023-01-07 16:33:15] ** Features with all missing measurements across runs are removed.
INFO  [2023-01-07 16:33:16] ** Shared peptides are removed.
INFO  [2023-01-07 16:33:16] ** Multiple measurements in a feature and a run are summarized by summaryforMultipleRows: max
INFO  [2023-01-07 16:33:17] ** Features with one or two meas

In [17]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_NN_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=4-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

554
92
1957
137
PPV = 0.857585
NPV = 0.934575


In [18]:
# quantms
msstats = pd.read_csv("./MSstats_quantms_NN_NaN_output_filter.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'SP=Yeast;CT=Mixture;QY=10-SP=Yeast;CT=Mixture;QY=1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

# positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
# negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

628
228
1821
63
PPV = 0.733645
NPV = 0.966561


In [10]:
evidata = pd.read_csv("../evidence.txt", sep="\t")
evidata = evidata[-((evidata["Reverse"] == "+") | (evidata["Potential contaminant"] == "+"))]

proteinGroups = pd.read_csv("../proteinGroups.txt", sep="\t")
proteinGroups = proteinGroups[-((proteinGroups["Reverse"] == "+") | (proteinGroups["Potential contaminant"] == "+"))]

evidata.head()

C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\IPython\core\interactiveshell.py:3296: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\IPython\core\interactiveshell.py:3296: DtypeWarning: Columns (166,167) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Phospho (STY) Probabilities,Oxidation (M) Score Diffs,Phospho (STY) Score Diffs,Acetyl (Protein N-term),Oxidation (M),...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,0,1697,0,0,0;1,0,NaN,NaN
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,1,1697,0,0,2,2,NaN,NaN
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,2,1697,0,0,3,3,NaN,NaN
3,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,3,1697,0,0,4,4,NaN,NaN
4,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,4,1697,0,0,5,5,NaN,NaN


In [35]:
print(len(set(evidata["Sequence"])))
print(len(set(evidata["Proteins"])))

15148
5771


In [18]:
filtered_df = evidata.groupby('Sequence').filter(lambda x: len(set(x["Proteins"])) == 1)
filtered_df = filtered_df.groupby('Protein group IDs').filter(lambda x: len(set(x["Sequence"])) >= 2)
filtered_df = filtered_df[-filtered_df["Proteins"].str.contains(";")]
filtered_df.head()

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Phospho (STY) Probabilities,Oxidation (M) Score Diffs,Phospho (STY) Score Diffs,Acetyl (Protein N-term),Oxidation (M),...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,0,1697,0,0,0;1,0,NaN,NaN
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,1,1697,0,0,2,2,NaN,NaN
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,2,1697,0,0,3,3,NaN,NaN
3,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,3,1697,0,0,4,4,NaN,NaN
4,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,4,1697,0,0,5,5,NaN,NaN


In [8]:
filtered_df.to_csv("../filter_evidence.txt", sep="\t", index=None)

In [20]:
print(len(set(filtered_df["Sequence"])))
print(len(set(filtered_df["Proteins"])))

12141
2987


In [25]:
out_msstats = pd.read_csv("../out_msstats_filter2.csv", index_col=None)

import re
def sub_mod(peptide):
    peptide = peptide.replace(".", "")
    peptide = re.sub("\(.*?\)", "", peptide)
    return peptide
    
out_msstats["sequence"] = out_msstats.apply(lambda x: sub_mod(x["PeptideSequence"]), axis=1)

print(len(set(out_msstats["sequence"])))
print(len(set(out_msstats["ProteinName"])))


out_msstats = pd.read_csv("../out_msstats.csv", index_col=None)
out_msstats["sequence"] = out_msstats.apply(lambda x: sub_mod(x["PeptideSequence"]), axis=1)
filtered_df = out_msstats.groupby('sequence').filter(lambda x: len(set(x["ProteinName"])) == 1)
filtered_df = filtered_df.groupby('ProteinName').filter(lambda x: len(set(x["sequence"])) >= 2)
filtered_df = filtered_df[-filtered_df["ProteinName"].str.contains(";")]
filtered_df.drop("sequence", axis=1, inplace=True)
filtered_df.to_csv("../out_msstats_filter2.csv", index=False)
print(len(set(out_msstats["sequence"])))
print(len(set(out_msstats["ProteinName"])))

11355
2773
13779
5127


In [9]:
proteinGroups = proteinGroups[proteinGroups["Unique peptides"] >= 2]
proteinGroups = proteinGroups[proteinGroups["Number of proteins"] == 1]
proteinGroups.head()

,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Fasta headers,Number of proteins,Peptides,Razor + unique peptides,Unique peptides,...,Peptide IDs,Peptide is razor,Mod. peptide IDs,Evidence IDs,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs,Oxidation (M) site positions,Phospho (STY) site positions
167,sp|A0AVK6|E2F8_HUMAN,sp|A0AVK6|E2F8_HUMAN,2,2,2,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,1,2,2,2,...,1456;5983,True;True,1835;7527,22404;22405;22406;22407;22408;86270,26689;26690;26691;26692;26693;103568,26691;103568,NaN,205;206,NaN,102;417
168,sp|A0FGR8|ESYT2_HUMAN,sp|A0FGR8|ESYT2_HUMAN,2,2,2,sp|A0FGR8|ESYT2_HUMAN Extended synaptotagmin-2...,1,2,2,2,...,870;11719,True;True,1098;1099;14903;14904;14905,13506;13507;13508;13509;13510;13511;13512;1351...,16072;16073;16074;16075;16076;16077;16078;1607...,16081;197908,18,207;208;209;210;211;212;213;214;215;16540;16541,716,688;691;693;699;701;704;705;736;737;738;739
170,sp|A0JNW5|UH1BL_HUMAN,sp|A0JNW5|UH1BL_HUMAN,5,5,5,sp|A0JNW5|UH1BL_HUMAN UHRF1-binding protein 1-...,1,5,5,5,...,1794;1795;11121;11661;12494,True;True;True;True;True,2260;2261;14107;14824;15906,27497;27498;27499;27500;27501;27502;27503;2750...,33486;33487;33488;33489;33490;33491;33492;3349...,33489;33499;185355;196577;210530,NaN,217;218;219;220;221;222;223,NaN,414;418;884;887;889;953;989
171,sp|A0MZ66|SHOT1_HUMAN,sp|A0MZ66|SHOT1_HUMAN,5,5,5,sp|A0MZ66|SHOT1_HUMAN Shootin-1 OS=Homo sapien...,1,5,5,5,...,2628;2701;12129;13616;14885,True;True;True;True;True,3336;3424;3425;15410;15411;17315;18916,38985;39843;39844;39845;39846;39847;39848;3984...,47030;48036;48037;48038;48039;48040;48041;4804...,47030;48055;203834;226653;246486,20,224;225;226;227;228;16542,507,492;494;506;534;537;619
174,sp|A1L390|PKHG3_HUMAN,sp|A1L390|PKHG3_HUMAN,3,3,3,sp|A1L390|PKHG3_HUMAN Pleckstrin homology doma...,1,3,3,3,...,4968;6263;15021,True;True;True,6266;7876;19074,72648;72649;72650;72651;72652;72653;72654;7265...,87994;87995;87996;87997;87998;87999;88000;8800...,87995;107156;248923,NaN,232;233;234,NaN,76;827;934


In [10]:
proteinGroups.to_csv("../filter_proteinGroups.txt", sep="\t", index=None)

In [11]:
%%R -w 800 -h 600

# peptide analyasis
library(MSstats)
library(reticulate)

# maxquant
#evidence
mq_ev = data.table::fread("../filter_evidence.txt")
#proteingroups
mq_pg = data.table::fread("../filter_proteinGroups.txt")
#annotation
annot = data.table::fread("../2-experimentalDesign.txt")
maxq_imported = MaxQtoMSstatsFormat(mq_ev, annot, mq_pg, use_log_file = FALSE)
fileData <- maxq_imported

# ---------------------

# If run dataProcess() occuring an error message, please change "summaryMethod = 'TMP'" to "summaryMethod = 'linear'"
DDA2009.proposed <- MSstats::dataProcess(raw = fileData,
                                         normalization = 'equalizeMedians',
                                         summaryMethod = 'TMP',
                                         censoredInt = "NA",
                                         MBimpute = TRUE)

# Automatically create the manually created matrix in MSstats, user manual p23
len <- length(levels(DDA2009.proposed$FeatureLevelData$GROUP))

tmp <- t(combn(len,2))
matrix_len = length(t(combn(len,2))) / 2

ourMatrix <- matrix(c(0:0),nrow=matrix_len,ncol=len)

for(i in 1:matrix_len){
  ourMatrix[i, tmp[i]] = -1
  ourMatrix[i, tmp[i + matrix_len]] = 1
}

ourCondition <- levels(DDA2009.proposed$ProteinLevelData$GROUP)
tmp_name <- matrix(ourCondition, nr=len, nc=1)
name <- matrix(nr=matrix_len, nc=1)
for(i in 1:matrix_len){
  name[i,1] <- sprintf('%s-%s', tmp_name[tmp[i+matrix_len]], tmp_name[tmp[i]])
}
row.names(ourMatrix) <- name

#----------End of creation-----------

colnames(ourMatrix) <- ourCondition

DDA2009.comparisons <- MSstats::groupComparison(contrast.matrix = ourMatrix,
                                                data = DDA2009.proposed)

write.csv(DDA2009.comparisons$ComparisonResult, file="MSstats_MaxQuant_filter_output.csv")

From cffi callback <function _consolewrite_ex at 0x0000027547BFA798>:
Traceback (most recent call last):
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\callbacks.py", line 132, in _consolewrite_ex
    s = conversion._cchar_to_str_with_maxlen(buf, n, _CCHAR_ENCODING)
  File "C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\rpy2\rinterface_lib\conversion.py", line 133, in _cchar_to_str_with_maxlen
    s = ffi.string(c, maxlen).decode(encoding)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd4 in position 1: invalid continuation byte
R[write to console]: The following object is masked from 'package:grDevices':

    savePlot




INFO  [2022-12-08 11:58:37] ** Raw data from MaxQuant imported successfully.
INFO  [2022-12-08 11:58:37] ** Rows with values of Potentialcontaminant equal to + are removed 
INFO  [2022-12-08 11:58:38] ** Rows with values of Reverse equal to + are removed 
INFO  [2022-12-08 11:58:38] ** Rows with values of Potentialcontaminant equal to + are removed 
INFO  [2022-12-08 11:58:38] ** Rows with values of Reverse equal to + are removed 
INFO  [2022-12-08 11:58:38] ** Rows with values of Onlyidentifiedbysite equal to + are removed 
INFO  [2022-12-08 11:58:38] ** + Contaminant, + Reverse, + Potential.contaminant, + Only.identified.by.site proteins are removed.
INFO  [2022-12-08 11:58:39] ** Raw data from MaxQuant cleaned successfully.
INFO  [2022-12-08 11:58:39] ** Using provided annotation.
INFO  [2022-12-08 11:58:39] ** Run labels were standardized to remove symbols such as '.' or '%'.
INFO  [2022-12-08 11:58:39] ** The following options are used:
  - Features will be defined by the columns:

### 4-1

In [12]:
# maxquant
msstats = pd.read_csv("./MSstats_MaxQuant_filter_output.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'fold4-fold1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.01) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.01) ]

no_difference_ups = ["P00167", "P01133", "P02144", "P04040", "P15559", "P62937", "P63165",  "Q06830"]
difference_ups =  ["P01579", "O76070", "P63279", "P68871","P01127", "P01008", "P62988", "P10599", "P02787", "P99999", "P12081", "P51965","P10636-8", 
       "P01031", "P09211", "P02788", "P41159", "O00762", "P05413", "P00441", "P00918", "P08758", "P00915", "P01344", "P69905", 
        "P00709", "P55957", "P08263", "P61769", "P10145", "P16083", "P61626", "P02741", "P06732",
        "P01375", "P06396", "P02753", "P01112", "Q15843", "P02768"]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

570
93
1883
68
PPV = 0.859729
NPV = 0.965146


## 10-1

In [13]:
# maxquant
msstats = pd.read_csv("./MSstats_MaxQuant_filter_output.csv", header=0, sep=",", index_col=0)
msstats = msstats[msstats['Label'] == 'fold10-fold1']
msstats.dropna(subset=["log2FC", "adj.pvalue"], how="any", inplace=True)

msstats["log2FC"] = pd.to_numeric(msstats["log2FC"],errors='coerce')

positive =  msstats[(abs(msstats["log2FC"]) >1) &(msstats["adj.pvalue"] <0.05) ]
negative = msstats[(abs(msstats["log2FC"]) <=1) | (msstats["adj.pvalue"] >=0.05) ]

no_difference_ups = ["P00167", "P01133", "P02144", "P04040", "P15559", "P62937", "P63165",  "Q06830"]
difference_ups =  ["P01579", "O76070", "P63279", "P68871","P01127", "P01008", "P62988", "P10599", "P02787", "P99999", "P12081", "P51965","P10636-8", 
       "P01031", "P09211", "P02788", "P41159", "O00762", "P05413", "P00441", "P00918", "P08758", "P00915", "P01344", "P69905", 
        "P00709", "P55957", "P08263", "P61769", "P10145", "P16083", "P61626", "P02741", "P06732",
        "P01375", "P06396", "P02753", "P01112", "Q15843", "P02768"]


TP = len(positive[positive["Protein"].str.contains("YEAST")])
FP = len(positive) - TP

FN = len(negative[negative["Protein"].str.contains("YEAST")])
TN = len(negative) - FN

print(TP)
print(FP)
print(TN)
print(FN)

print("PPV = %.6f" % (TP / (TP + FP)))
print("NPV = %.6f" % (TN / (TN + FN)))

619
342
1634
19
PPV = 0.644121
NPV = 0.988506


## Missing values evalute

In [39]:
evidata = pd.read_csv("../evidence.txt", sep="\t")
evidata = evidata[-((evidata["Reverse"] == "+") | (evidata["Potential contaminant"] == "+"))]
evidata = evidata.groupby('Sequence').filter(lambda x: len(set(x["Proteins"])) == 1)
evidata = evidata.groupby('Protein group IDs').filter(lambda x: len(set(x["Sequence"])) >= 2)
evidata = evidata[-evidata["Proteins"].str.contains(";")]
evidata

C:\Users\ChengXin\MyProgram\python\anaconda\pro\lib\site-packages\IPython\core\interactiveshell.py:3296: DtypeWarning: Columns (56) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Phospho (STY) Probabilities,Oxidation (M) Score Diffs,Phospho (STY) Score Diffs,Acetyl (Protein N-term),Oxidation (M),...,Reverse,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,0,1697,0,0,0;1,0,NaN,NaN
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,1,1697,0,0,2,2,NaN,NaN
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,2,1697,0,0,3,3,NaN,NaN
3,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,3,1697,0,0,4,4,NaN,NaN
4,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,NaN,4,1697,0,0,5,5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215972,YYSDSDDELTVEQRRQSIAK,20,Phospho (STY),_YYSDS(Phospho (STY))DDELTVEQRRQSIAK_,NaN,YYS(0.002)DS(0.998)DDELTVEQRRQSIAK,NaN,Y(-71)Y(-66)S(-28)DS(28)DDELT(-34)VEQRRQS(-53)IAK,0,0,...,NaN,NaN,215972,4257,15382,19533,255289,255289,NaN,12146;12147
215973,YYSDSDDELTVEQRRQSIAK,20,Phospho (STY),_YYSDS(Phospho (STY))DDELTVEQRRQSIAK_,NaN,YYS(0.045)DS(0.952)DDELT(0.002)VEQRRQS(0.002)IAK,NaN,Y(-53)Y(-48)S(-13)DS(13)DDELT(-28)VEQRRQS(-28)IAK,0,0,...,NaN,NaN,215973,4257,15382,19533,255290,255290,NaN,12146;12147
215974,YYSDSDDELTVEQRRQSIAK,20,Phospho (STY),_YYS(Phospho (STY))DSDDELTVEQRRQSIAK_,NaN,YYS(0.992)DS(0.008)DDELTVEQRRQSIAK,NaN,Y(-37)Y(-37)S(21)DS(-21)DDELT(-44)VEQRRQS(-48)IAK,0,0,...,NaN,NaN,215974,4257,15382,19533,255291,255291,NaN,12146;12147
215975,YYSDSDDELTVEQRRQSIAK,20,Phospho (STY),_YYS(Phospho (STY))DSDDELTVEQRRQSIAK_,NaN,Y(0.002)Y(0.002)S(0.942)DS(0.047)DDELT(0.003)V...,NaN,Y(-27)Y(-27)S(13)DS(-13)DDELT(-25)VEQRRQS(-25)IAK,0,0,...,NaN,NaN,215975,4257,15382,19533,255292,255292,NaN,12146;12147


In [41]:
evidata["Condition"] = evidata.apply(lambda x: x["Experiment"].split("_")[0], axis=1)
evidata.head()

,Sequence,Length,Modifications,Modified sequence,Oxidation (M) Probabilities,Phospho (STY) Probabilities,Oxidation (M) Score Diffs,Phospho (STY) Score Diffs,Acetyl (Protein N-term),Oxidation (M),...,Potential contaminant,id,Protein group IDs,Peptide ID,Mod. peptide ID,MS/MS IDs,Best MS/MS,Oxidation (M) site IDs,Phospho (STY) site IDs,Condition
0,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,0,1697,0,0,0;1,0,NaN,NaN,10
1,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,1,1697,0,0,2,2,NaN,NaN,10
2,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,2,1697,0,0,3,3,NaN,NaN,10
3,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,3,1697,0,0,4,4,NaN,NaN,1
4,AAAAAAAAAPAAAATAPTTAATTAATAAQ,29,Unmodified,_AAAAAAAAAPAAAATAPTTAATTAATAAQ_,NaN,NaN,NaN,NaN,0,0,...,NaN,4,1697,0,0,5,5,NaN,NaN,1


In [49]:
missing_number = 0
for c, c_group in evidata.groupby(["Condition", "Sequence"]):
    missing_number = missing_number + 6 - len(set(c_group["Experiment"]))
    if c[1] == "LVLAESPQPK":
        print(6 - len(set(c_group["Experiment"])))
print(missing_number / (len(set(evidata["Sequence"]))*6))

0
0
0
0.6118935837245696


In [48]:
out_msstats = pd.read_csv("C:/Users/ChengXin/MyProgram/quantms/PXD007145/PXD007145.sdrf_openms_design_msstats_in.csv", index_col=None)
out_msstats["sequence"] = out_msstats.apply(lambda x: sub_mod(x["PeptideSequence"]), axis=1)
filtered_df = out_msstats.groupby('sequence').filter(lambda x: len(set(x["ProteinName"])) == 1)
filtered_df = filtered_df.groupby('ProteinName').filter(lambda x: len(set(x["sequence"])) >= 2)
filtered_df = filtered_df[-filtered_df["ProteinName"].str.contains(";")]

missing_number = 0
for c, c_group in filtered_df.groupby(["Condition", "sequence"]):
    missing_number = missing_number + 6 - len(set(c_group["Run"]))

print(missing_number / (len(set(out_msstats["sequence"]))*6))

0.42694196482570096
